- Precisa fazer lazy uncompress e verify
- Precisa fazer tambem lazy evaluation dos passos internos para detalhar

In [1]:
from adapt_utils import replace_symbols
from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps

from tree_parser import *

import networkx as nx

import matplotlib
import matplotlib.pyplot as plt

import copy

import time

from collections import Counter

In [2]:
%%time

text = file_contents()
database = meta_math_database(text,n=3000)
print("")

included 5555695 tokens from set.mm
proposition: 3000
Wall time: 9.64 s


In [37]:
class file_contents:
    def __init__(self):
        self.included_files = set()
        self.tokens = []
        self.input_file(initial_file_name)
        self.current_index = 0

    # read the contents of filename, splits into space-separated list,
    # and appends to self.tokens.
    def input_file(self, filename):
        full_path = os.path.realpath(filename)
        if full_path in self.included_files:
            return
        f = open(full_path, 'r')
        f_contents = f.read().split()
        f.close()
        print("included", len(f_contents), "tokens from", filename)
        f_contents.reverse()
        self.tokens += f_contents
        self.included_files.add(full_path)

    def read_token(self):
        if len(self.tokens) == 0: return None
        return self.tokens.pop()

    def read_until(self,end_token):
        out = []
        in_comment = False
        while True:
            next_token = self.read_token()
            assert next_token != None
            if next_token==end_token: return out
            if next_token=='$(': in_comment=True
            if not in_comment: out.append(next_token)
            if next_token=='$)': in_comment=False
                
    def go_to_prop(self, prop):
        for i, t in enumerate(self.tokens):
            if t == prop and "$" in self.tokens[i-1]: #need to check i-1 because tokens here are reversed
                self.tokens = self.tokens[:i]
                return
            
        print(f"Prop {prop} not found")

In [51]:
text = file_contents()
text.go_to_prop("dfss2")

included 5555695 tokens from set.mm


In [ ]:
def parse_p(self, current_block, file_contents):
    if self.remember_proof_steps:
        statement = file_contents.read_until("$=")
        proof = file_contents.read_until("$.")
        prop = proposition('p', self.label, current_block,statement,proof,number=len(self.propositions),)
        self.propositions[self.label]=prop
        self.propositions_list.append(prop)

        self.label = None

        #return

        self.uncompress(prop,current_block)  # uncompress the proof
        prop.update_optional_hypotheses(current_block)
        if len(self.propositions) % 10 ==0:#print len(self.propositions)#print prop.label #prop.print_details()
            sys.stdout.write('\rproposition: '+str(len(self.propositions)))
            sys.stdout.flush()


        #if statement[0]!="|-":
        #    self.non_entails_axioms[prop.label]=prop # these are all provable from non $p statements, so...
        # try to verify the proposition
        self.verify(prop)

    # don't remember steps.  don't verify.
    else:
        #constant = file_contents.read_token()
        statement = file_contents.read_until("$.")
        if statement[0]=="|-":
            _, tree = self.parser.parse_new_statement(["wff"]+statement[1:],current_block)
        else:
            tree = Tree(value=self.label,leaves=[f.tree for f in current_block.hyps if f.variable in statement])
            #print '\nunparsed axiom', self.label, statement, tree.stringify()
        prop = proposition('p',self.label,current_block,statement,None,number=len(self.propositions), tree=tree)
        self.propositions[self.label]=prop
        self.propositions_list.append(prop)
        if len(self.propositions) % 10 ==0:#print len(self.propositions)#print prop.label #prop.print_details()
            sys.stdout.write('\rproposition: '+str(len(self.propositions)))
            sys.stdout.flush()
        self.label = None